# Imports

This is imported in the different notebooks of the repository and provides common functions for the analysis and determination of the workspaces of electromagnetic navigation systems.

In [1]:
from ipywidgets import *
import os.path

# Magnetic models

from mag_manip.mag_manip import *

# Graphics and plotting

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import cm
import matplotlib as mpl
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation
from IPython import display

from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Maths and computation

import numpy as np
import math
import yaml
from numpy.linalg import lstsq
import pandas as pd

from scipy.spatial.transform import Rotation as R
from scipy.linalg import null_space
from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from scipy.optimize import linprog

from itertools import combinations
from itertools import product

# Interval analysis

from interval import interval 
from interval import imath
from interval import fpu
from pyibex import *

# Utility functions for the workspace analysis and evaluation of eMNS

## Functions for feasibility test with zonotopes

Approach proposed in [Bouchard 2008](https://asmedigitalcollection.asme.org/mechanismsrobotics/article/2/1/011010/475593/On-the-Ability-of-a-Cable-Driven-Robot-to-Generate) to check if required field can be generated in one position.

In [2]:
#Compute the vertices of the convex hull defining the zonotope
def ComputeZonotopeVertices(Imin,Imax,J):
    # Imin: minimum current (scalar) in A
    # Imax: maximum current (scalar) in A
    # J: Jacobian matrix of the eMNS
    # vert: coordinates of the vertices of the zonotope of the available set of tasks

    N = np.shape(J)[1]
    pts = np.stack(([0,1],)*N,0)
    A_alpha = np.transpose((np.array(np.meshgrid(*pts)).T).reshape(2**N,N)) #compute permutation matrix
    
    M = np.matmul(J,(Imax - Imin)*np.eye(N))
    C = np.matmul(M,A_alpha)
    
    hull = ConvexHull(np.transpose(C))
    hull_idx = hull.vertices
    C_hull = C[:,hull_idx]
    
    vert = np.transpose(C_hull + np.matmul(J,Imin*np.ones((N,1))))
    
    return vert

#Create permutation matrix for the selection of unitary actuation fields
def CreatePermuationMatrix(A):
    # A: Jacobian matrix
    # M: permutation matrix
    
    d = np.shape(A)[0] #dimension of output space (if field, this is 3)
    n = np.shape(A)[1] #number of coils
    
    comb = combinations(np.arange(n), d-1) 

    M = np.asarray(list(comb))  
    
    return M

#Create combination matrix to test combination of field
def CreateFieldCombinationMatrix(n):
    # n: dimension of the combination matrix
    # M: combination matrix

    nums = np.arange(2**n)
    M = ((nums.reshape(-1,1) & (2**np.arange(n))) != 0).astype(int)
    
    return M

#Compute the hyperplane representation of the zonotope
def HyperPlaneShiftingMethod(A,Imin,Imax):
    # Imin: minimum current (scalar) in A
    # Imax: maximum current (scalar) in A
    # J: Jacobian matrix of the eMNS
    # N, d_vec: Hyperplane representation of the zonotope

    dI = Imax - Imin
    M = CreatePermuationMatrix(A)
    nb_comb = np.shape(M)[0] #number of combination
    
    d = np.shape(A)[0] #dimension of output space (if field, this is 3)
    n_coils = np.shape(A)[1] #number of coils
    
    #Initialize matrix and vector for hyperplane representation
    N = np.zeros((2*nb_comb,d))
    d_vec = np.zeros((2*nb_comb,1))
    
    bmin = np.matmul(A,Imin*np.ones((n_coils,1)))
    
    #Iterate on the combination of unitary fields
    for i in range(nb_comb):
        
            
        # Step 1: define initial hyperplane
        
        #Define the set of vectors to be orthogonal with
        W = A[:,M[i,:]]
        
        #Get the orthogonal vector using the nullspace of W^T
        Wns = null_space(np.transpose(W))
        v = Wns[:,0]

        # Step 2: shift intial hyperplane
        temp = v / np.linalg.norm(v)
        n = temp.reshape((-1,1))
        
        # Step 3: build projections   
        lj_arr = np.zeros((n_coils-(d-1),1))
        k = 0
        h = 0. 
        for j in range(n_coils):
            if not(j in M[i,:]):
                lj = np.dot(np.transpose(A[:,j]),n)
                lj_arr[k,0] = lj
                k += 1

        C = CreateFieldCombinationMatrix(n_coils-(d-1))
        
        h = np.matmul(C,dI*lj_arr)
        
        hp = np.max(h)
        hm = np.min(h)
        
        #Step 4: compute hyperplane support
        pp = hp*n + bmin
        pm = hm*n + bmin
        
        
        # Step 5: build hyperplane representation
        N[i,:] = n.T
        N[i+nb_comb,:] = -n.T
        d_vec[i,:] = np.dot(n.T,pp)
        d_vec[i+nb_comb,:] = np.dot(-n.T,pm)
            
        
    return N, d_vec

#Check if a given task field set of the form of a convex polytope complies with the available field zonotope
def VerifyFeasabilityPolytope(d,N,V):
    # d, N: hyperplane representation of the zonotope
    # V: vertices of the polytope of the task field
    # M: matrix of feasbility for the desired tasks
    # isFeasible: overall feasibility for the set of tasks

    
    D = np.repeat(d,np.shape(V)[1],axis=1)
    M = np.matmul(N,V) <= D
    isFeasible = np.all(M)
    
    return M, isFeasible

#Check if a given task field of the form of a point complies with the available field zonotope
def VerifyFeasabilityPoint(d,N,b):
    # d, N: hyperplane representation of the zonotope
    # b: desired field tasks
    # M: matrix of feasbility for the desired tasks
    # isFeasible: overall feasibility for the set of tasks

    M = np.matmul(N,b) <= d
    isFeasible = np.all(M)
    
    return M, isFeasible

#Check if a given task field of the form of an ellipsoid complies with the available field zonotope
def VerifyFeasabilityEllipsoid(d,N,a):
    # d, N: hyperplane representation of the zonotope
    # a: the half axes of the task field expressed as an ellipsoid
    # M: matrix of feasbility for the desired tasks
    # isFeasible: overall feasibility for the set of tasks

    #Assume feasibility a priori
    isFeasible = True

    for i in range(N.shape[0]):
        
        n = np.transpose(N[i,:])
        kinv = 0
        
        for j in range(N.shape[1]):
            kinv = kinv + (a[j]*n[j])**2
            
        k = 1 / math.sqrt(kinv)
        
        e_p = np.matmul(np.diag(a**2)*k, n)
        e_m = np.matmul(-np.diag(a**2)*k, n)
        
        #Test feasibility in the directions of the ellipsoid normals colinear to the hyperplanes of the available set
        M, isFeasible_p = VerifyFeasabilityPoint(d,N,e_p)
        M, isFeasible_m = VerifyFeasabilityPoint(d,N,e_m)
        
        #Return false if at least one of the hyperplane is infeasible
        if not(isFeasible_p) or not(isFeasible_m):
            isFeasible = False
            break

    return isFeasible

# Check if a matrix a is invertible
def is_invertible(a):
    return a.shape[0] == a.shape[1] and np.linalg.matrix_rank(a) == a.shape[0]

### Functions for the convex hull computation of a zonotope

In [3]:
#Class to compute order faces from a convex hull
class Faces():
    def __init__(self,tri, sig_dig=12, method="convexhull"):
        self.method=method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms,return_inverse=True, axis=0)

    def norm(self,sq):
        cr = np.cross(sq[2]-sq[0],sq[1]-sq[0])
        return np.abs(cr/np.linalg.norm(cr))

    def isneighbor(self, tr1,tr2):
        a = np.concatenate((tr1,tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0))+2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n,v[1]-v[0])
        y = y/np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1]-v[0],y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c,axis=0)
            d = c-mean
            s = np.arctan2(d[:,0], d[:,1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j,tri2 in enumerate(self.tri):
                if j > i: 
                    if self.isneighbor(tri1,tri2) and \
                       self.inv[i]==self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups

#Plot the convec hull from a set a points 
def plotHull(ax,bmax,color,alpha,plot_vertices=False,set_edges=True,inter = 0.02,edge_color = 'white'):
    # ax: plot axes
    # bmax: set of points to determine the convex hull from
    # color: color of the hull
    # alpha: transparency of the hull
    # plot_vertices: plot the vertices [True] or not [False]
    # set_edges: plot the edges of the hull [True] or not [False]
    # inter: interval of the majo locator on the axes
    # edge_color: color of the edges (only if set_edges is True)
    
    
    hull = ConvexHull(bmax)
    simplices = hull.simplices
    vert_idx = hull.vertices
    org_triangles = [bmax[s] for s in simplices]

    f = Faces(org_triangles)
    g = f.simplify()
    ax.scatter(bmax[:,0],bmax[:,1], bmax[:,2], color=color,alpha=0.) #plot corners with transparency to ensure proper scale
    if plot_vertices:
        ax.scatter(bmax[vert_idx,0],bmax[vert_idx,1], bmax[vert_idx,2], color='blue',alpha=1.)
    pc = Poly3DCollection(g,facecolor=color, alpha=alpha)
    if set_edges:
        pc.set_edgecolor(edge_color)
    ax.add_collection3d(pc)
    ax.xaxis.set_major_locator(MultipleLocator(inter))
    ax.yaxis.set_major_locator(MultipleLocator(inter))
    ax.zaxis.set_major_locator(MultipleLocator(inter))

## Functions for evaluation metrics

In [4]:
#Compute the manipulability index as defines by Yoshikawa
def ComputeYoshikawaManipulability(A,amax): 
    # A: Jacobian matrix
    # amax: maximum values of the inputs
    # mu: manipulability index

    if not np.shape(amax):
        amax_vec = np.repeat(amax, A.shape[0])
        S = np.diag(1/amax_vec)
    else:
        S = np.diag(1/amax)

    An = np.matmul(S,A)

    mu = np.sqrt(np.linalg.det(np.matmul(An,np.transpose(An))))

    return mu

#Compute the condition number
def ComputeConditionNumber(A,amax):
    # A: Jacobian matrix
    # amax: maximum values of the inputs
    # k: condition number

    if not np.shape(amax):
        amax_vec = np.repeat(amax, A.shape[0])
        S = np.diag(1/amax_vec)
    else:
        S = np.diag(1/amax)

    An = np.matmul(S,A)

    u, s, v= np.linalg.svd(An)

    w_min  = np.min(s)
    w_max  = np.max(s)

    k = w_max/w_min

    return k

#Minimum distance to boundaries of the available set (depends on the desired task) also called DA distance
def MinDistanceToA(d,N,DesTask,task_type="polytope"):
    # d, N: hyperplane representation of the zonotope
    # task_type: 
    #   "polytope": DesTask defines the vetices of the polytope
    #   "point": DeskTask defines the individual task
    #   "ellipsoid": DesTask defines the semi-axes length of the ellipsoid
    # minDist: compute to minimum distance to the desired task to the zonotope's hyperplanes


    if task_type == "polytope":
        V = DesTask
        D = np.repeat(d,np.shape(V)[1],axis=1)
        M = D - np.matmul(N,V) 
        minDist = M.min()
    elif task_type == "point":
        b = DesTask
        D = d
        M = D - np.matmul(N,b)
        minDist = M.min()
    elif task_type == "ellipsoid":
        a = DesTask
        D = d
        minDist = 1e10 #high init of min dist
        
        for i in range(N.shape[0]):

            n = np.transpose(N[i,:])
            kinv = 0

            for j in range(N.shape[1]):
                kinv = kinv + (a[j]*n[j])**2

        k = 1 / math.sqrt(kinv)

        e_p = np.matmul(np.diag(a**2)*k, n)
        e_m = np.matmul(-np.diag(a**2)*k, n)

        M = D - np.matmul(N,e_p)
        if M.min() < minDist:
            minDist = M.min()
        M = D - np.matmul(N,e_m)
        if M.min() < minDist:
            minDist = M.min()    
                
                
    else:
        print("The required type of task set does not exist!")
        return  


    return minDist

## Functions for workspace determination

In [5]:
# Determine the 2d workspace of a eMNS with discretization method
def Ws2DDeterminationDiscr(Nx,Ny,pmin,pmax,bmin,bmax,Imin,Imax,model_mns):
    # Nx: number of points along x-direction
    # Ny: number of points along y-direction
    # pmin: minimum position
    # pmax: maximum position
    # bmin: minimum magnetic field to achieve
    # bmax: maximum magnetic field to achieve
    # Imin: minimum current
    # Imax: maximum current
    # model_mns: model of the eMNS
    # p_all: tested positions
    # p_ws_in: positions in the ws
    # p_ws_out: positions out of the ws
    # p_ws_side: positions on the side of the tested area
    # kappa: condition number at the positions in the ws
    # mu: manpulability index at the positions in the ws
    # gci: global conditionning index of the ws
    # minDist: DA distance at the position in the ws

    p_ws_in = np.empty((0,3))
    p_ws_out = np.empty((0,3))
    p_ws_side = np.empty((0,3))
    p_all = np.empty((0,3))
    
    kappa = np.array([])
    mu = np.array([])
    minDist = np.array([])
    
    index_sum = 0
    total_pts = 0

    posx_v = np.linspace(pmin, pmax, Nx, endpoint=True)
    posy_v = np.linspace(pmin, pmax, Ny, endpoint=True)
    posz_v = np.linspace(0, 0, 1, endpoint=True)

    #Define task polytope
    V = np.array([[bmin, bmax, bmax, bmin],[bmin, bmin, bmax, bmax]])

    for i in range(posx_v.shape[0]):
        for j in range(posy_v.shape[0]):
            for k in range(posz_v.shape[0]):

                x = posx_v[i]
                y = posy_v[j]
                z = posz_v[k]
                
                position = np.array([x,y,z])
                Jtemp = model_mns.getFieldActuationMatrix(position)
                J = Jtemp[0:2,:]

                #Check polytope feasibility at each grid point            
                N, d = HyperPlaneShiftingMethod(J,Imin,Imax)
                M, isFeasible = VerifyFeasabilityPolytope(d,N,V)
                
                p_all = np.append(p_all, [position], axis=0)

                if isFeasible:
                    mu_val = ComputeYoshikawaManipulability(J,bmax)
                    kappa_val = ComputeConditionNumber(J,bmax)
                    mindist_val = MinDistanceToA(d, N, V)
                    mu = np.append(mu,[mu_val], axis=0)
                    kappa = np.append(kappa,[kappa_val], axis=0)
                    minDist = np.append(minDist,[mindist_val], axis=0)
                    p_ws_in = np.append(p_ws_in, [position], axis=0)
                    index_sum += 1/kappa_val
                    total_pts += 1
                else:
                    p_ws_out = np.append(p_ws_out, [position], axis=0) 
                    
                
                if x == pmin or x == pmax or y == pmin or y == pmax:
                    p_ws_side = np.append(p_ws_side, [position], axis=0)
                    
    gci = index_sum/total_pts
    
    return p_all, p_ws_in, p_ws_out, p_ws_side, kappa, mu, gci, minDist

#Compute the set of position where a task field set can be generated
#(require at least 3 coils)
def Ws3DFieldDeterminationDiscr(Nx,Ny,Nz,pmin,pmax,bmin,bmax,Imin,Imax,model_mns,task_type="polytope"):
    # Nx: number of points along x-direction
    # Ny: number of points along y-direction
    # Nz: number of points along z-direction
    # pmin: minimum position
    # pmax: maximum position
    # bmin: minimum magnetic field to achieve
    # bmax: maximum magnetic field to achieve
    # Imin: minimum current
    # Imax: maximum current
    # model_mns: model of the eMNS
    # task_type: type of the desired task (polytope, or ellipsoid)
    # p_ws_in: positions in the ws
    # p_ws_out: positions out of the ws
    # p_ws_side: positions on the side of the tested area
    # kappa: condition number at the positions in the ws
    # mu: manpulability index at the positions in the ws
    # gci: global conditionning index of the ws
    # minDist: DA distance at the position in the ws

    
    p_ws_in = np.empty((0,3))
    p_ws_out = np.empty((0,3))
    p_ws_side = np.empty((0,3))

    posx_v = np.linspace(pmin[0], pmax[0], Nx, endpoint=True)
    posy_v = np.linspace(pmin[1], pmax[1], Ny, endpoint=True)
    posz_v = np.linspace(pmin[2], pmax[2], Nz, endpoint=True)
    
        
    kappa = np.array([])
    mu = np.array([])
    minDist = np.array([])
    
    index_sum = 0
    total_pts = 0

    
    if task_type == "polytope":
        #Define task polytope
        V = np.transpose(bmin + (bmax-bmin)*CreateFieldCombinationMatrix(3))
    elif task_type == "ellipsoid":
        #Define task ellipse (circle)
        a = np.array([bmax, bmax, bmax])
    else:
        print("The required type of task set does not exist!")
        return
        
    for i in range(posx_v.shape[0]):
        for j in range(posy_v.shape[0]):
            for k in range(posz_v.shape[0]):        
                                
                x = posx_v[i]
                y = posy_v[j]
                z = posz_v[k]

                position = np.array([posx_v[i],posy_v[j],posz_v[k]])
                J = model_mns.getFieldActuationMatrix(position)

                #Check feasibility at each grid point            
                N, d = HyperPlaneShiftingMethod(J,Imin,Imax)
                
                if task_type == "polytope":
                    M, isFeasible = VerifyFeasabilityPolytope(d,N,V)
                    DesTask = V
                elif task_type == "ellipsoid":
                    isFeasible = VerifyFeasabilityEllipsoid(d,N,a)
                    DesTask = a

                
                if isFeasible and np.linalg.matrix_rank(J) >= 3:
                    p_ws_in = np.append(p_ws_in, [position], axis=0)
                    mu_val = ComputeYoshikawaManipulability(J,bmax)
                    kappa_val = ComputeConditionNumber(J,bmax)
                    minDist_val = MinDistanceToA(d, N, DesTask, task_type)
                    mu = np.append(mu,[mu_val], axis=0)
                    kappa = np.append(kappa,[kappa_val], axis=0)
                    minDist = np.append(minDist,[minDist_val], axis=0)
                    index_sum += 1/kappa_val
                    total_pts += 1
                else:
                    p_ws_out = np.append(p_ws_out, [position], axis=0) 
                    
                if (x == pmin[0] or x == pmax[0]) and (y == pmin[1] or y == pmax[1]):
                    p_ws_side = np.append(p_ws_side, [position], axis=0)
                    
    gci = index_sum/total_pts
    
    return p_ws_in, p_ws_out, p_ws_side, kappa, mu, gci, minDist



#Compute the set of position where a task gradient set can be generated
#(require at least 5 coils)
def Ws3DGradientDeterminationDiscr(Nx,Ny,Nz,pmin,pmax,gmin,gmax,Imin,Imax,model_mns,task_type="polytope"):
    # Nx: number of points along x-direction
    # Ny: number of points along y-direction
    # Nz: number of points along z-direction
    # pmin: minimum position
    # pmax: maximum position
    # gmin: minimum magnetic field gradient to achieve
    # gmax: maximum magnetic field gradient to achieve
    # Imin: minimum current
    # Imax: maximum current
    # model_mns: model of the eMNS
    # task_type: type of the desired task (polytope, or ellipsoid)
    # p_ws_in: positions in the ws
    # p_ws_out: positions out of the ws
    # p_ws_side: positions on the side of the tested area
    # kappa: condition number at the positions in the ws
    # mu: manpulability index at the positions in the ws
    # gci: global conditionning index of the ws
    # minDist: DA distance at the position in the ws

    p_ws_in = np.empty((0,3))
    p_ws_out = np.empty((0,3))
    p_ws_side = np.empty((0,3))

    posx_v = np.linspace(pmin[0], pmax[0], Nx, endpoint=True)
    posy_v = np.linspace(pmin[1], pmax[1], Ny, endpoint=True)
    posz_v = np.linspace(pmin[2], pmax[2], Nz, endpoint=True)
    
    kappa = np.array([])
    mu = np.array([])
    minDist = np.array([])
    
    index_sum = 0
    total_pts = 0

    #Define task
    if task_type == "polytope":
        #Define task polytope
        V = np.transpose(gmin + (gmax-gmin)*CreateFieldCombinationMatrix(5))
    elif task_type == "ellipsoid":
        #Define task ellipse (circle)
        a = np.array([gmax, gmax, gmax, gmax, gmax])
    else:
        print("The required type of task set does not exist!")
        return
    
    for i in range(posx_v.shape[0]):
        for j in range(posy_v.shape[0]):
            for k in range(posz_v.shape[0]):
                
                x = posx_v[i]
                y = posy_v[j]
                z = posz_v[k]

                position = np.array([posx_v[i],posy_v[j],posz_v[k]])
                Jtemp = model_mns.getActuationMatrix(position)
                J = Jtemp[3:8,:]

                #Check feasibility at each grid point            
                N, d = HyperPlaneShiftingMethod(J,Imin,Imax)
                
                if task_type == "polytope":
                    M, isFeasible = VerifyFeasabilityPolytope(d,N,V)
                    DesTask = V
                elif task_type == "ellipsoid":
                    isFeasible = VerifyFeasabilityEllipsoid(d,N,a)
                    DesTask = a

                if isFeasible and np.linalg.matrix_rank(J) >= 5:
                    p_ws_in = np.append(p_ws_in, [position], axis=0)
                    mu_val = ComputeYoshikawaManipulability(J,gmax)
                    kappa_val = ComputeConditionNumber(J,gmax)
                    minDist_val = MinDistanceToA(d, N, DesTask, task_type)
                    mu = np.append(mu,[mu_val], axis=0)
                    kappa = np.append(kappa,[kappa_val], axis=0)
                    minDist = np.append(minDist,[minDist_val], axis=0)
                    index_sum += 1/kappa_val
                    total_pts += 1
                else:
                    p_ws_out = np.append(p_ws_out, [position], axis=0) 
                    
                                
                if (x == pmin[0] or x == pmax[0]) and (y == pmin[1] or y == pmax[1]):
                    p_ws_side = np.append(p_ws_side, [position], axis=0)
                    
    gci = index_sum/total_pts
    
    return p_ws_in, p_ws_out, p_ws_side, kappa, mu, gci, minDist


#Compute the set of position where a task field and gradient set can be generated
#(only applicable to 8-coil-systems)
def Ws3DFieldGradientDeterminationDiscr(Nx,Ny,Nz,pmin,pmax,bmin,bmax,gmin,gmax,Imin,Imax,model_mns,task_type="polytope"):
    # Nx: number of points along x-direction
    # Ny: number of points along y-direction
    # Nz: number of points along z-direction
    # pmin: minimum position
    # pmax: maximum position
    # bmin: minimum magnetic field to achieve
    # bmax: maximum magnetic field to achieve
    # gmin: minimum magnetic field gradient to achieve
    # gmax: maximum magnetic field gradient to achieve
    # Imin: minimum current
    # Imax: maximum current
    # model_mns: model of the eMNS
    # task_type: type of the desired task (polytope, or ellipsoid)
    # p_ws_in: positions in the ws
    # p_ws_out: positions out of the ws
    # p_ws_side: positions on the side of the tested area
    # kappa: condition number at the positions in the ws
    # mu: manpulability index at the positions in the ws
    # gci: global conditionning index of the ws
    # minDist: DA distance at the position in the ws
    
    
    p_ws_in = np.empty((0,3))
    p_ws_out = np.empty((0,3))
    p_ws_side = np.empty((0,3))

    posx_v = np.linspace(pmin[0], pmax[0], Nx, endpoint=True)
    posy_v = np.linspace(pmin[1], pmax[1], Ny, endpoint=True)
    posz_v = np.linspace(pmin[2], pmax[2], Nz, endpoint=True)
    
    kappa = np.array([])
    mu = np.array([])
    minDist = np.array([])
    
    index_sum = 0
    total_pts = 0
    
    
    #Define task
    if task_type == "polytope":
        #Define task polytope
        Mcomb = np.transpose(CreateFieldCombinationMatrix(8))
        Vfield = bmin + (bmax-bmin)*Mcomb[0:3,:]
        Vgradient = gmin + (gmax-gmin)*Mcomb[4:8,:]
        V = np.vstack(Vield,Vgradient)
        amax = np.concatenate((bmax,gmax))
    elif task_type == "ellipsoid":
        #Define task ellipse (circle)
        a = np.array([bmax, bmax, bmax, gmax, gmax, gmax, gmax, gmax])
        amax = a
    else:
        print("The required type of task set does not exist!")
        return
      

    for i in range(posx_v.shape[0]):
        for j in range(posy_v.shape[0]):
            for k in range(posz_v.shape[0]):
                
                 
                x = posx_v[i]
                y = posy_v[j]
                z = posz_v[k]

                position = np.array([posx_v[i],posy_v[j],posz_v[k]])
                J = model_mns.getActuationMatrix(position)

                #Check feasibility at each grid point            
                N, d = HyperPlaneShiftingMethod(J,Imin,Imax)
                
                if task_type == "polytope":
                    M, isFeasible = VerifyFeasabilityPolytope(d,N,V)
                    DesTask = V
                elif task_type == "ellipsoid":
                    isFeasible = VerifyFeasabilityEllipsoid(d,N,a)
                    DesTask = a

                if isFeasible and is_invertible(J):
                    p_ws_in = np.append(p_ws_in, [position], axis=0)
                    mu_val = ComputeYoshikawaManipulability(J,amax)
                    kappa_val = ComputeConditionNumber(J,amax)
                    minDist_val = MinDistanceToA(d, N, DesTask, task_type)
                    mu = np.append(mu,[mu_val], axis=0)
                    kappa = np.append(kappa,[kappa_val], axis=0)
                    minDist = np.append(minDist,[minDist_val], axis=0)
                    index_sum += 1/kappa_val
                    total_pts += 1
                else:
                    p_ws_out = np.append(p_ws_out, [position], axis=0) 
                    
                                    
                if (x == pmin[0] or x == pmax[0]) and (y == pmin[1] or y == pmax[1]):
                    p_ws_side = np.append(p_ws_side, [position], axis=0)
                    
    gci = index_sum/total_pts
    
    return p_ws_in, p_ws_out, p_ws_side, kappa, mu, gci, minDist

## Function for coils vizualisation

In [6]:
# Plot 2D coils
def Coils2DPatch(coil_length,coil_width,cal_path_mns):
    # coil_length: length of the coils representation
    # coil_width: widfth of the coils representation
    # cal_path_mns: path to the calibration file of the eMNS
    
    yaml_file = open(cal_path_mns)
    parsed_yaml_file = yaml.load(yaml_file, Loader=yaml.FullLoader)
    
    patches_list = []
    
    coil_list = parsed_yaml_file.get("Coil_List")
    n_coil = len(coil_list)
    
    #Iterate on coils
    for i in range(0,n_coil):
        
        #Get actual coil
        coil_i = parsed_yaml_file.get("Coil_"+str(i))
        n_sources = len(coil_i.get("Source_List"))
        

        #Iterate on sources for the coil
        for j in range(0,n_sources):

            #Get source parameters
            src_j = coil_i.get("Src_"+str(j))                           
            src_dir = src_j.get("Source_Direction")
            src_pos = src_j.get("Source_Position")

            #Transform position in source frame
            xpos = src_pos[0]
            ypos = src_pos[1]
            zpos = src_pos[2]
            
            angle = np.arctan2(src_dir[1],src_dir[0])
            
            #Coil
            coil = patches.Rectangle((0,0), coil_length,coil_width, color="black",  alpha=0.80)
            toff = mpl.transforms.Affine2D().translate(-coil_length/2,-coil_width/2)
            r = mpl.transforms.Affine2D().rotate_deg(angle*180/math.pi) 
            t = mpl.transforms.Affine2D().translate(xpos,ypos)
            coil.set_transform(toff + r + t)
            
            #Center
            center = patches.Circle((xpos, ypos), coil_width/10, color="blue",  alpha=1.)
            
            patches_list.append(coil)
            patches_list.append(center)

    
    p = PatchCollection(patches_list, alpha=0.4)
    
    return p

# Plot 3D coils
def PlotCoils3D(length,radius,cal_path_mns,ax,DoRaster=False):
    # length: length of the coils representation
    # radius: radius of the coils representation
    # cal_path_mns: path to the calibration file of the eMNS
    # ax: axis of the figure to represent the coils
    # DoRaster: rasterize the representation [True] or not [False]
    
    yaml_file = open(cal_path_mns)
    parsed_yaml_file = yaml.load(yaml_file, Loader=yaml.FullLoader)
    
    patches_list = []
    
    coil_list = parsed_yaml_file.get("Coil_List")
    n_coil = len(coil_list)
    
    #Iterate on coils
    for i in range(0,n_coil):
        
        #Get actual coil
        coil_i = parsed_yaml_file.get("Coil_"+str(i))
        n_sources = len(coil_i.get("Source_List"))
        

        #Iterate on sources for the coil
        for j in range(0,n_sources):

            #Get source parameters
            src_j = coil_i.get("Src_"+str(j))                           
            src_dir = src_j.get("Source_Direction")
            src_pos = src_j.get("Source_Position")

            x_center = src_pos[0]
            y_center = src_pos[1]
            z_center = src_pos[2]
            
            v_dir = np.array([src_dir[0], src_dir[1], src_dir[2]]) #dir
            
            #axis and radius
            p0 = np.array([x_center, y_center, z_center]) - length/2*v_dir #point at one end
            p1 = np.array([x_center, y_center, z_center]) + length/2*v_dir #point at other end

            #vector in direction of axis
            v = p1 - p0

            #find magnitude of vector
            mag = np.linalg.norm(v)

            #unit vector in direction of axis
            v = v / mag

            #make some vector perpendicular to v
            not_v_mat = np.array([[-v[1],v[0],0.],[-v[2],0.,v[0]],[0.,v[2],-v[1]]])
            norm_vec = np.linalg.norm(not_v_mat, axis=1)
            not_v = not_v_mat[np.argmax(norm_vec),:]

            #make vector perpendicular to v
            n1 = np.cross(v, not_v)
            #normalize n1
            n1 /= np.linalg.norm(n1)

            #make unit vector perpendicular to v and n1
            n2 = np.cross(v, n1)

            #surface ranges over t from 0 to length of axis and 0 to 2*pi
            t = np.linspace(0, mag, 2)
            theta = np.linspace(0, 2 * np.pi, 70)
            rsample = np.linspace(0, radius, 2)

            #use meshgrid to make 2d arrays
            t, theta2 = np.meshgrid(t, theta)

            rsample,theta = np.meshgrid(rsample, theta)

            #generate coordinates for surface
            # "Tube"
            X, Y, Z = [p0[i] + v[i] * t + radius * np.sin(theta2) * n1[i] + radius * np.cos(theta2) *       n2[i] for i in [0, 1, 2]]
            # "Bottom"
            X2, Y2, Z2 = [p0[i] + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]
            # "Top"
            X3, Y3, Z3 = [p0[i] + v[i]*mag + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]

            ax.plot_surface(X, Y, Z, color='#e39e21ff',  alpha=1.0, rasterized=DoRaster)
            ax.plot_surface(X2, Y2, Z2, color="#e39e21ff",  alpha=1.0, rasterized=DoRaster)
            ax.plot_surface(X3, Y3, Z3, color="#e39e21ff",  alpha=1.0, rasterized=DoRaster)

    return

## Functions for interval analysis

### Intervals evaluation and workspace determination

In [7]:
# Interval evaluation of the norm inorm of an interval vector [xi, yi, zi]
def IntervalEvaluateVectorNorm(xi,yi,zi):

    # Compute norm
    inorm = imath.sqrt(xi**2 + yi**2 + zi**2)
    
    return inorm

# Interval evaluation of the normalize vector [xni, yni, zni] from a vector [xi, yi, zi]
def IntervalEvaluateNormalizedVector(xi,yi,zi):

    # Compute norm
    inorm = IntervalEvaluateVectorNorm(xi,yi,zi)
    
    # Compute component intervals
    xni = (xi / inorm) & interval[-1.,1.]
    yni = (yi / inorm) & interval[-1.,1.]
    zni = (zi / inorm) & interval[-1.,1.]

    return xni, yni, zni

# Interval evaluation of the Legendre polynomial P and its derivative Pp of order n at x
def IntervalEvaluateLegendrePolyNomial(x,n):
    
    #LegendrePoly and derivative for order 0 to 2
    
    if n == 0:
        P = 1
        Pp = 0
    elif n == 1:
        P = x
        Pp = 1 
    elif n == 2:
        P = 0.5 * (3*x**2 - 1)
        Pp = 3*x
        
    return P, Pp

# Interval evaluation of the actuation matrix Blist of a system defined in parsed_yaml_file in the interval [xi, yi, zi]
def IntervalEvaluateActuationMatrix(xi,yi,zi,parsed_yaml_file):
    
    coil_list = parsed_yaml_file.get("Coil_List")
    n_coil = len(coil_list)
    
    #Initialize interval actuation matrix
    Blist = [[0 for c in range(n_coil)] for r in range(2)]
    
    #Iterate on coils
    for i in range(0,n_coil):
        
        #Get actual coil
        coil_i = parsed_yaml_file.get("Coil_"+str(i))
        n_sources = len(coil_i.get("Source_List"))
        
        #Initialize coil contrib terms
        Bcoil_x = 0.
        Bcoil_y = 0.
        Bcoil_z = 0.

        #Iterate on sources for the coil
        for j in range(0,n_sources):

            #Get source parameters
            src_j = coil_i.get("Src_"+str(j))                         
            A = src_j.get("A_Coeff") #0 outside of the coils
            B = src_j.get("B_Coeff")  
            src_dir = src_j.get("Source_Direction")
            src_pos = src_j.get("Source_Position")

            #Transform position in source frame
            xs = (xi - src_pos[0])
            ys = (yi - src_pos[1])
            zs = (zi - src_pos[2])
            
            #Get norm from position vector for intervals
            
            rnorm = IntervalEvaluateVectorNorm(xs,ys,zs)
            xni, yni, zni = IntervalEvaluateNormalizedVector(xs,ys,zs)
            
            #Interval evaluation of cos from inclination angle
            gamma = (src_dir[0] * xni) + (src_dir[1] * yni) + (src_dir[2] * zni)
            
            #Consider only the dipole term
            P, Pp = IntervalEvaluateLegendrePolyNomial(gamma,1)
            C1 = - (2*B[0] / (rnorm ** 3))
            C2 = B[0] / (rnorm ** 3)
            
            
            #Simplified expression
            Bsrc_x = B[0] * (3 * gamma * xni - src_dir[0]) / (rnorm ** 3)
            Bsrc_y = B[0] * (3 * gamma * yni - src_dir[1]) / (rnorm ** 3)
            
            #Add source contribution to coil
            Bcoil_x = Bcoil_x + Bsrc_x
            Bcoil_y = Bcoil_y + Bsrc_y
    
        Blist[0][i] = Bcoil_x
        Blist[1][i] = Bcoil_y

    
    return Blist

### "Feasible" and "Out" routines

In [8]:
# Test if the box is feasible
def Feasible(Ainter,Imin,Imax, bmin, bmax):
    # Ainter: the interval actuation matrix
    # Imin and Imax: the min and max available current
    # bmin, bmax: minimum and maximum field to achieve
    
    d = np.shape(Ainter)[0]
    
    #Build the vertex matrices

    #Create combination matrix
    M = CreateFieldCombinationMatrix(d)
    Ytemp = 2*M-1
    Y = Ytemp.transpose()
    
    #print(Y)
    #print("\n")
    
    
    #Initialize feasability to True
    isFeasible = True
    
    #print(Ainter)
    #print("\n")
    
    for k in range(2**d):        
        #Initialization of the vertex matrix and vertex vector
        Ay = [[0 for c in range(len(Ainter[0]))] for r in range(d)]
        Amy = [[0 for c in range(len(Ainter[0]))] for r in range(d)]
        by = np.zeros((d,1))
        
        #Contruction
        for i in range(d):
            for j in range(len(Ainter[0])):
                Aij = Ainter[i][j]
                Ay[i][j] = Aij[0][0] + (Aij[0][1] - Aij[0][0])*(1 - Y[i,k])/2
                Amy[i][j] = Aij[0][0] + (Aij[0][1] - Aij[0][0])*(1 + Y[i,k])/2
                
       
            by[i] = bmin + (bmax - bmin) * (1 + Y[i,k])/2
        
        
        #Check strong solvability at the vertex using simplex method
        n = np.shape(Ainter)[1]
        c = np.zeros((2*n,1))
        A = np.concatenate((np.asarray(Ay), -np.asarray(Amy)), axis=1)
        b = by
        
        Imax_v = Imax * np.ones((n,1))
        Imin_v = -Imin * np.ones((n,1))
        beq = np.concatenate((Imax_v, Imin_v), axis=0)
        I = np.eye(n)
        Aeq_temp_up = np.concatenate((I, -I), axis=1)
        Aeq_temp_down = np.concatenate((-I, I), axis=1)
        Aeq = np.concatenate((Aeq_temp_up, Aeq_temp_down), axis=0)
        
        bnds = (0, None)
        
        try:
            res = linprog(c, Aeq, beq, A, b, bounds=(bnds))
            isVertexFeasible = res.success
        except:
            isVertexFeasible = False

        
        if not(isVertexFeasible):
            isFeasible = False
            break

    
    return isFeasible

# Test if the box is out of the workspace
def Out(Ainter,Imin,Imax, bmin, bmax):
    # Ainter: the interval actuation matrix
    # Imin and Imax: the min and max available current
    # bmin, bmax: minimum and maximum field to achieve
    
    isOut = False
    
    d = np.shape(Ainter)[0]
    nc = len(Ainter[0])
    
    #Fetch the right function
    strDim = str(d) 
    strCoil = str(nc)  
    
    func_type = strDim + "D" + strCoil + "Coils.txt"
    
    f = Function(func_type)
    
    #Build contractor for equality constraint
    ctc = CtcFwdBwd(f, EQ)
             
    #Build interval matrix and interval vector
    i_list = []
    Ainter_list =[]
    #Build intervals

    for i in range(d):   
        for j in range(nc):
            Aij = Ainter[i][j]
            Ainter_list.append(Interval(Aij[0][0],Aij[0][1]))

    for k in range(nc):
        i_list.append(Interval(Imin,Imax))
    
    #Create combination matrix
    M = CreateFieldCombinationMatrix(d)
    Ytemp = 2*M-1
    Y = Ytemp.transpose()

    for k in range(2**d):        

        by_list = []

        #Contruction
        for i in range(d):
            byi = bmin + (bmax - bmin) * (1 + Y[i,k])/2
            by_list.append(Interval(byi,byi))

        #Build interval vector
        X = IntervalVector(tuple(i_list) + tuple(by_list) + tuple(Ainter_list))
        ctc.contract(X)
        
        #if one of the contracted interval is empty, the box is out
        if X.is_empty():
            isOut = True
            break

    return isOut

### Bisection and workspace computation methods

In [9]:
# Create a 2D box for interval analysis
def Create2DBox(xi,yi):
    # xi, yi: boundary of the box in x and y direction

    #Initilialize
    Box = [[0 for c in range(1)] for r in range(2)]
    comb = product(np.arange(2),repeat=2)
    M = np.asarray(list(comb))  

    xmin = xi[0][0]
    xmax = xi[0][1]
    ymin = yi[0][0]
    ymax = yi[0][1]
    Box[0][0] = interval[xmin, xmax] 
    Box[1][0] = interval[ymin, ymax] 
    
    return Box

# Bisection of a box and return the list of bisected boxes
def BisectBox(Box):

    d = np.shape(Box)[0]

    #Initilialize
    Bb = [[0 for c in range(2)] for r in range(d)]
    comb = product(np.arange(2),repeat=d)
    M = np.asarray(list(comb))  

    #Divide each 1D-intervals
    for i in range(d): 
        inter = Box[i][0]
        xmin = inter[0][0]
        xmax = inter[0][1]
        Bb[i][0] = interval[xmin, (xmin+xmax)/2] 
        Bb[i][1] = interval[(xmin+xmax)/2, xmax] 

    #Create list of bisected boxes
    #Box list
    Box_list = []
    for i in range(2**d):
        B_temp = [[0 for c in range(1)] for r in range(d)]
        for j in range(d):  
            B_temp[j][0] = Bb[j][M[i,j]]
        #print(B_temp)
        Box_list.append(B_temp)
    
    return Box_list

# Determination of the 2D worskpace of a eMNS using interval analysis
def Ws2DDetermination(InitBox, bmin, bmax, Imin, Imax, eps, parsed_yaml_file,zoff,max_iter=-1):
    # InitBox: initial box
    # Imin and Imax: the min and max available current
    # bmin, bmax: minimum and maximum field to achieve
    # eps: size of the smallest box to finish bissecting
    # parsed_yaml_file: calibration file of the eMNS
    # zoff: offset in z of the system
    # max_iter[optional]: maximum number of iteration (default is infinite)
    # L: list of boxes
    # Lin: list of boxes in the ws
    # Lout: list of boxes out of the ws
    # Lneg: list of undetermined boxes
    # iteration: number of iterations
    
    L = []
    Lin = []
    Lout = []
    Lneg = []
    L.append(InitBox)
    iteration = 0
        
    while(len(L)>0):
        #Extract intervals
        CurrentBox = L[0]
        del L[0] #remove the element from the list
        #Build interval matrix
        xi = CurrentBox[0][0]
        yi = CurrentBox[1][0]
        Ainter = IntervalEvaluateActuationMatrix(xi,yi,zoff,parsed_yaml_file)
        #Check feasbility
        isFeasible = Feasible(Ainter,Imin,Imax,bmin,bmax)

        if isFeasible:
            Lin.append(CurrentBox)
            #print('in')
        else:
            isOut = Out(Ainter,Imin,Imax,bmin,bmax)
            #isOut = False
            if isOut:
                Lout.append(CurrentBox)
                #print('out')
            else: 
                if (xi[0][1] - xi[0][0]) > eps and (yi[0][1] - yi[0][0]) > eps:
                    Bisected_box = BisectBox(CurrentBox)
                    L.extend(Bisected_box)
                else:
                    Lneg.append(CurrentBox)  
        iteration = iteration + 1
                
        if iteration >= max_iter and max_iter >= 1:
            print('Max iter reached')
            break
    
    return L, Lin, Lout, Lneg, iteration